In [0]:
!git clone https://github.com/16cs009/alyceproject.git
import os
os.chdir("/content/alyceproject")
!pip install -r requirements.txt

from transformers.modeling_gpt2 import GPT2LMHeadModel
_ = GPT2LMHeadModel.from_pretrained("gpt2-medium")

!pip install "python-socketio[client]"


import numpy as np
import torch
import plot_generator
from Datasets import datasets
tokenizer, model, PPLM = plot_generator.initialize(np,torch)

In [0]:
def phaseII_processStory(storylist):
	# Returns a list of list with the following as the content of list inside the list :>
	# 1. Dialogue-Replaced sentence from story or Generated PLOT
	# 2. Type of the Sentence : NARRATION, DIALOGUE, NEWLINE, PLOT
	# 3. YES or NO string indicating whether NeuralCoref was applied to resolve reference
	# 4. If NeuralCoref was applied then The object It returned
	# 5. If the sentence is of type dialogue then its speaker
	# 6. If the sentence is of type dialogue then the corresponding dialogue
	plotwords = []
	tmp = list()
	new_storylist = []
	tmp_storylist = []
	p_count = 0
	for sentences in storylist:
		tmp_storylist.append(sentences)
		if sentences[1]=="NEWLINE":
			if len(tmp_storylist)>0:
				new_storylist.append([p_count,"PLOT","","","",""])
				p_count = p_count + 1
			new_storylist += tmp_storylist
			tmp_storylist = []
			plotwords.append(tmp)
			tmp = list()
		elif sentences[1]=="NARRATIVE":
			tmp = tmp + sentences[0].split(" ")
		else:
			tmp = tmp + sentences[0].split(" ")
			tmp = tmp + sentences[5].split(" ")
	#print(new_storylist)
	for i in range(len(plotwords)):
		plotwords[i] = list(dict.fromkeys([ ("".join([tmpchar for tmpchar in word if tmpchar.isalpha()==True])).lower() for word in plotwords[i] if word.find("<-DIALOGUE-#")==-1 and len(word)>2 and word not in datasets.wordList]))
		plotwords[i].sort()
	plot_desc = []
	print("Required to genearte "+str(len(plotwords)+1)+" words!...")
	for i in plotwords:
		if len(i)>0:
			print("Generating Plot!....")
			plot_desc.append(plot_generator.generate_plot(np,torch,tokenizer,model,PPLM,[i],20,1)[0])
			print("Generating Plot Completed!....")
		else:
			plot_desc.append('')
	new_storylist = [sentences if sentences[1]!="PLOT" else [plot_desc[sentences[0]],"PLOT","","","",""] for sentences in new_storylist]
	return new_storylist

In [0]:
def phaseII_processStory_Step_1(storylist):
	# Returns a new_storylist and plotwords
	plotwords = []
	tmp = list()
	new_storylist = []
	tmp_storylist = []
	p_count = 0
	for sentences in storylist:
		tmp_storylist.append(sentences)
		if sentences[1]=="NEWLINE":
			if len(tmp_storylist)>0:
				new_storylist.append([p_count,"PLOT","","","",""])
				p_count = p_count + 1
			new_storylist += tmp_storylist
			tmp_storylist = []
			plotwords.append(tmp)
			tmp = list()
		elif sentences[1]=="NARRATIVE":
			tmp = tmp + sentences[0].split(" ")
		else:
			tmp = tmp + sentences[0].split(" ")
			tmp = tmp + sentences[5].split(" ")
	#print(new_storylist)
	for i in range(len(plotwords)):
		plotwords[i] = list(dict.fromkeys([ ("".join([tmpchar for tmpchar in word if tmpchar.isalpha()==True])).lower() for word in plotwords[i] if word.find("<-DIALOGUE-#")==-1 and len(word)>2 and word not in datasets.wordList]))
		plotwords[i].sort()
	new_storylist = [sentences if sentences[1]!="PLOT" else ["","PLOT",plotwords[sentences[0]],[],"",""] for sentences in new_storylist]
	return new_storylist

"""
def phaseII_processStory_Step_2(plotwords):
	# Returns a list of strings(PLOTS) as the content.
	plot_desc = []
	print("Required to genearte "+str(len(plotwords)+1)+" words!...")
	for i in plotwords:
		if len(i)>0:
			print("Generating Plot!....")
			plot_desc.append(plot_generator.generate_plot(np,torch,tokenizer,model,PPLM,[i],20,1)[0])
			print("Generating Plot Completed!....")
		else:
			plot_desc.append('')
	return plot_desc
"""

def phaseII_processStory_Step_2(plotwords):
	# Returns a list of strings(PLOTS) as the content.
	plot_desc = []
	print("Required to genearte "+str(len(plotwords)+1)+" words!...")
	for i in plotwords:
		if len(i[1])>0:
			print("Generating Plot!....")
			tmp = []
			tmp.append(i[0])
			tmp.append(plot_generator.generate_plot(np,torch,tokenizer,model,PPLM,[i[1]],20,1)[0])
			plot_desc.append(tmp)
			print("Generating Plot Completed!....")
		else:
			plot_desc.append([i[0],''])
	return plot_desc

In [0]:
import socketio
import json
import time

sio = socketio.Client()

"""def workHandler(input_data):
	sio.emit('phase-2-server-to-main-server',{"client": input_data["client"], "procedureNo": input_data["procedureNo"], "status": "STARTED", "output": json.dumps([["None"]])})
	output_data = phaseII_processStory(input_data["input"])
	#sio.sleep(30);
	sio.emit('phase-2-server-to-main-server',{"client": input_data["client"], "procedureNo": input_data["procedureNo"], "status": "COMPLETED", "output": json.dumps(output_data)})"""

def workHandler(input_data):
	sio.emit('phase-2-server-to-main-server',{"client": input_data["client"], "procedureNo": input_data["procedureNo"], "status": "STARTED", "output": json.dumps([["None"]])})
	output_data = []
	if(input_data["procedureNo"]==0):
		output_data = phaseII_processStory_Step_1(input_data["input"])
	elif(input_data["procedureNo"]==1):
		output_data = phaseII_processStory_Step_2(input_data["input"])
	#sio.sleep(30);
	print("Return Data : "+str(json.dumps(output_data)))
	sio.emit('phase-2-server-to-main-server',{"client": input_data["client"], "procedureNo": input_data["procedureNo"], "status": "COMPLETED", "output": json.dumps(output_data)})

@sio.on('main-server-to-phase-2-server')
def on_phaseTwoData(data):
	print("RECEIVED DATA : {}".format(data))
	sio.start_background_task(workHandler, data)

print("Starting Sub server of Phase 2 !...")
sio.connect(input("Enter NGROK Link : "))
sio.emit('server-ready',{"phase": 2})
print("Started Sub server of Phase 2 !...")

try:
	while True:
		time.sleep(1)
except KeyboardInterrupt:
	print("Interrupt Received!...")
	sio.disconnect()

print("Exiting Sub Server!....")